## Definindo os objetivos específicos da análise:

- Analisar os dados úteis;
- Tratar os dados;
- Descartar os dados inúteis para análise;
- Criar resumos, nos quais digam as ideias principais do arquivo e suas palavra chaves;
- Teste da eficácia do programa/códigos;
- Fazer as mudanças necessárias para melhor eficiência.

## Objetivos dos códigos: 

- Criar uma função que visualize e identifique apenas os dados úteis para fazer resumos contábeis utilizando modelos de linguagens;

- Criar uma função que descarte os arquivos inúteis e notifique isso;

- Criar resumos claros e completos dos dados de fácil visualização;

- Fazer testes para verificar a eficiência dos resumos, para poder melhorar ele.

In [ ]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import os
from typing import List, Optional

# Tentar baixar stopwords se não estiverem presentes
try:
    stopwords.words('portuguese')
except LookupError:
    nltk.download('stopwords')
try:
    word_tokenize("exemplo")
except LookupError:
    nltk.download('punkt')

# Carregar o modelo de linguagem para spaCy
try:
    nlp = spacy.load("pt_core_news_sm")
except OSError:
    print("Downloading pt_core_news_sm language model for spaCy...")
    spacy.cli.download("pt_core_news_sm")
    nlp = spacy.load("pt_core_news_sm")

def identificar_dados_uteis_contabeis(
    df: pd.DataFrame, colunas_texto: Optional[List[str]] = None
) -> pd.DataFrame:
    """
    Visualiza e identifica dados úteis para resumos contábeis em um DataFrame.

    Args:
        df (pd.DataFrame): DataFrame contendo os dados contábeis.
        colunas_texto (List[str], optional): Lista de colunas que contêm texto para análise.
                                         Se None, todas as colunas string/object serão consideradas.
                                         Defaults to None.

    Returns:
        pd.DataFrame: DataFrame com informações sobre a utilidade de cada coluna.
    """
    info_utilidade = {}
    for col in df.columns:
        info = {"tipo": str(df[col].dtype), "n_na": df[col].isna().sum(), "n_unique": df[col].nunique()}

        if pd.api.types.is_numeric_dtype(df[col]):
            info["utilidade"] = "Potencialmente útil (numérico)"
            info["exemplos"] = df[col].dropna().head().to_list()
        elif pd.api.types.is_datetime64_any_dtype(df[col]):
            info["utilidade"] = "Potencialmente útil (data)"
            info["exemplos"] = df[col].dropna().head().to_list()
        elif pd.api.types.is_string_dtype(df[col]) or df[col].dtype == "object":
            info["exemplos"] = df[col].dropna().astype(str).head().to_list()
            if colunas_texto is None or col in colunas_texto:
                textos = " ".join(df[col].dropna().astype(str).head(100).tolist())
                doc = nlp(textos)
                entidades = [ent.text for ent in doc.ents]
                palavras_chave = [token.text for token in doc if token.is_alpha and not token.is_stop]

                if entidades or any(
                    palavra in ["R$", "USD", "valor", "total", "conta", "débito", "crédito", "imposto", "data", "ano"]
                ):
                    info["utilidade"] = "Altamente útil (texto contábil)"
                elif info["n_unique"] / len(df) > 0.8:
                    info["utilidade"] = "Baixa utilidade (alta cardinalidade)"
                elif len(set(palavras_chave)) < 5:
                    info["utilidade"] = "Baixa utilidade (pouca informação textual)"
                else:
                    info["utilidade"] = "Potencialmente útil (texto)"
            else:
                info["utilidade"] = "Não especificado para análise de texto"
        else:
            info["utilidade"] = "Outro tipo"
            info["exemplos"] = df[col].dropna().head().to_list()

        info_utilidade[col] = info

    return pd.DataFrame.from_dict(info_utilidade, orient="index")


def descartar_arquivos_inuteis(
    lista_arquivos: List[str], palavras_chave_uteis: Optional[List[str]] = None
) -> List[str]:
    """
    Descartar arquivos considerados inúteis com base em seus nomes e notifica isso.

    Args:
        lista_arquivos (List[str]): Lista de nomes de arquivos a serem avaliados.
        palavras_chave_uteis (List[str], optional): Lista de palavras-chave que indicam
                                                    que um arquivo pode ser útil. Defaults to None.

    Returns:
        List[str]: Lista de arquivos considerados úteis.
    """
    arquivos_uteis = []
    arquivos_descartados = []

    if palavras_chave_uteis is None:
        palavras_chave_uteis = [
            "contabil",
            "financeiro",
            "balanco",
            "dre",
            "fluxo",
            "caixa",
            "fiscal",
            "imposto",
        ]

    for arquivo in lista_arquivos:
        nome_lower = arquivo.lower()
        if any(keyword in nome_lower for keyword in palavras_chave_uteis):
            arquivos_uteis.append(arquivo)
        else:
            arquivos_descartados.append(arquivo)

    if arquivos_descartados:
        print(
            "Os seguintes arquivos foram considerados inúteis e descartados: "
            f"{', '.join(arquivos_descartados)}"
        )

    return arquivos_uteis


def criar_resumos_claros(df: pd.DataFrame, colunas_resumo: List[str], num_sentencas: int = 3) -> str:
    """
    Cria resumos claros e completos dos dados de fácil visualização.

    Args:
        df (pd.DataFrame): DataFrame contendo os dados a serem resumidos.
        colunas_resumo (List[str]): Lista de colunas de texto a serem usadas para o resumo.
        num_sentencas (int, optional): Número de sentenças desejado no resumo. Defaults to 3.

    Returns:
        str: Resumo dos dados.
    """
    textos_combinados = ""
    for col in colunas_resumo:
        if col in df.columns and (
            pd.api.types.is_string_dtype(df[col]) or df[col].dtype == "object"
        ):
            textos_combinados += " ".join(df[col].dropna().astype(str).tolist()) + " "

    if not textos_combinados.strip():
        return "Nenhum texto relevante encontrado para gerar o resumo."

    # Pré-processamento básico
    textos_combinados = textos_combinados.lower()
    tokens = word_tokenize(textos_combinados)
    stop_words = set(stopwords.words("portuguese"))
    tokens_filtrados = [w for w in tokens if not w in stop_words and w.isalnum()]
    texto_processado = " ".join(tokens_filtrados)

    # Vetorização TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([texto_processado])

    # Calcular a importância das sentenças (simples, baseado na frequência de termos)
    sentencas = nltk.sent_tokenize(textos_combinados)
    if not sentencas:
        return "Não foram encontradas sentenças para resumir."

    sentenca_pontuacao = {}
    for i, sentenca in enumerate(sentencas):
        tokens_sentenca = [word.lower() for word in word_tokenize(sentenca) if word.isalnum()]
        pontuacao = 0
        for token in tokens_sentenca:
            if token in vectorizer.vocabulary_:
                indice = vectorizer.vocabulary_[token]
                pontuacao += tfidf_matrix[0, indice]
        sentenca_pontuacao[i] = pontuacao

    # Selecionar as melhores sentenças para o resumo
    melhores_sentencas_indices = sorted(
        sentenca_pontuacao, key=sentenca_pontuacao.get, reverse=True
    )[:num_sentencas]
    melhores_sentencas_indices.sort()  # Manter a ordem original

    resumo = " ".join([sentencas[i] for i in melhores_sentencas_indices])
    return f"**Resumo Contábil:**\n\n{resumo}"


def testar_eficiencia_resumo(resumo_gerado: str, resumo_esperado: str) -> float:
    """
    Realiza um teste simples para verificar a eficiência do resumo gerado.
    Utiliza similaridade de cossenos entre as representações TF-IDF dos resumos.

    Args:
        resumo_gerado (str): O resumo gerado pela função.
        resumo_esperado (str): Um resumo de referência (feito por um humano, por exemplo).

    Returns:
        float: Uma pontuação de similaridade (entre 0 e 1), onde 1 indica alta similaridade.
    """
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([resumo_gerado, resumo_esperado])
    similarity_score = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]
    return similarity_score


def extrair_dados_relevantes(
    df: pd.DataFrame, colunas_relevantes: List[str]
) -> pd.DataFrame:
    """
    Extrai apenas as colunas relevantes de um DataFrame.

    Args:
        df (pd.DataFrame): DataFrame original.
        colunas_relevantes (List[str]): Lista de nomes de colunas a serem extraídas.

    Returns:
        pd.DataFrame: Novo DataFrame contendo apenas as colunas relevantes.
    """
    try:
        df_relevante = df[colunas_relevantes].copy()  # Use .copy() para evitar warnings
        return df_relevante
    except KeyError as e:
        print(f"Erro: Coluna não encontrada - {e}")
        return pd.DataFrame()  # Retorna um DataFrame vazio em caso de erro


def tratar_valores_ausentes(
    df: pd.DataFrame, estrategia: str = "preencher_zero"
) -> pd.DataFrame:
    """
    Trata valores ausentes em um DataFrame usando uma estratégia especificada.

    Args:
        df (pd.DataFrame): DataFrame com valores ausentes.
        estrategia (str, optional): Estratégia para tratar valores ausentes.
            Opções: 'preencher_zero', 'remover_linhas', 'media', 'mediana'.
            Defaults to 'preencher_zero'.

    Returns:
        pd.DataFrame: DataFrame com valores ausentes tratados.
    """
    df_tratado = df.copy()  # Cria uma cópia para não modificar o DataFrame original diretamente
    if estrategia == "preencher_zero":
        df_tratado.fillna(0, inplace=True)
    elif estrategia == "remover_linhas":
        df_tratado.dropna(inplace=True)
    elif estrategia == "media":
        for col in df_tratado.columns:
            if pd.api.types.is_numeric_dtype(df_tratado[col]):
                media = df_tratado[col].mean()
                df_tratado[col].fillna(media, inplace=True)
    elif estrategia == "mediana":
        for col in df_tratado.columns:
            if pd.api.types.is_numeric_dtype(df_tratado[col]):
                mediana = df_tratado[col].median()
                df_tratado[col].fillna(mediana, inplace=True)
    else:
        print("Estratégia inválida. Usando 'preencher_zero' por padrão.")
        df_tratado.fillna(0, inplace=True)
    return df_tratado


def normalizar_dados(df: pd.DataFrame, metodo: str = "zscore") -> pd.DataFrame:
    """
    Normaliza os dados numéricos de um DataFrame usando um método especificado.

    Args:
        df (pd.DataFrame): DataFrame com dados numéricos.
        metodo (str, optional): Método de normalização. Opções: 'zscore', 'minmax'.
            Defaults to 'zscore'.

    Returns:
        pd.DataFrame: DataFrame com os dados normalizados.
    """
    df_normalizado = df.copy()  # Cria cópia para não alterar o original
    for col in df_normalizado.columns:
        if pd.api.types.is_numeric_dtype(df_normalizado[col]):
            if metodo == "zscore":
                media = df_normalizado[col].mean()
                desvio_padrao = df_normalizado[col].std()
                if desvio_padrao == 0:
                    print(
                        f"Aviso: Desvio padrão é 0 para a coluna '{col}'. Pulando a normalização Z-score."
                    )
                    continue  # Pula para a próxima coluna
                df_normalizado[col] = (df_normalizado[col] - media) / desvio_padrao
            elif metodo == "minmax":
                min_val = df_normalizado[col].min()
                max_val = df_normalizado[col].max()
                if max_val - min_val == 0:
                    print(
                        f"Aviso: Amplitude é 0 para a coluna '{col}'. Pulando a normalização Min-Max."
                    )
                    continue  # Pula para a próxima coluna
                df_normalizado[col] = (df_normalizado[col] - min_val) / (
                    max_val - min_val
                )
            else:
                print("Método de normalização inválido. Usando 'zscore' por padrão.")
                media = df_normalizado[col].mean()
                desvio_padrao = df_normalizado[col].std()
                if desvio_padrao == 0:
                    print(
                        f"Aviso: Desvio padrão é 0 para a coluna '{col}'. Pulando a normalização Z-score."
                    )
                    continue  # Pula para a próxima coluna
                df_normalizado[col] = (df_normalizado[col] - media) / desvio_padrao
    return df_normalizado


def analisar_e_salvar_dados(
    nome_arquivo_csv: str,
    lista_arquivos: List[str],
    colunas_relevantes: List[str],
    estrategia_tratamento_ausentes: str = "preencher_zero",
    metodo_normalizacao: str = "zscore",
    colunas_resumo: Optional[List[str]] = None,
    num_sentencas_resumo: int = 3,
    resumo_esperado: Optional[str] = None,
) -> None:
    """
    Função principal que coordena o processo de análise, resumo e salvamento dos dados.

    Args:
        nome_arquivo_csv (str): Nome do arquivo CSV para salvar os resultados.
        lista_arquivos (List[str]): Lista de arquivos a serem processados.
        colunas_relevantes (List[str]): Colunas a serem extraídas para análise.
        estrategia_tratamento_ausentes (str, optional): Estratégia para tratar valores ausentes.
            Defaults to 'preencher_zero'.
        metodo_normalizacao (str, optional): Método de normalização. Defaults to 'zscore'.
        colunas_resumo (List[str], optional): Colunas para gerar o resumo. Se None, usa todas as colunas de texto.
            Defaults to None.
        num_sentencas_resumo (int, optional): Número de sentenças no resumo. Defaults to 3.
        resumo_esperado (str, optional): Resumo de referência para teste de eficiência. Defaults to None.
    """
    # Descartar arquivos inúteis
    arquivos_uteis = descartar_arquivos_inuteis(lista_arquivos)
    print(f"\nArquivos considerados úteis: {arquivos_uteis}")

    # Carregar os dados do arquivo (assumindo o primeiro arquivo útil como exemplo)
    if not arquivos_uteis:
        print("Erro: Nenhum arquivo útil encontrado para processar.")
        return

    # Verificar se o arquivo existe antes de tentar carregá-lo
    if not os.path.exists(arquivos_uteis[0]):
        print(f"Erro: O arquivo '{arquivos_uteis[0]}' não foi encontrado.")
        return

    try:
        df = pd.read_csv(arquivos_uteis[0])  # Carrega o primeiro arquivo útil
    except Exception as e:
        print(f"Erro ao carregar o arquivo CSV: {e}")
        return

    # Extrair dados relevantes
    df_relevante = extrair_dados_relevantes(df, colunas_relevantes)
    if df_relevante.empty:
        print("Erro: DataFrame resultante está vazio após a extração de colunas.")
        return

    # Tratar valores ausentes
    df_tratado = tratar_valores_ausentes(df_relevante, estrategia_tratamento_ausentes)

    # Normalizar dados
    df_normalizado = normalizar_dados(df_tratado, metodo_normalizacao)

    # Analisar a utilidade das colunas
    if colunas_resumo is None:
        colunas_texto_para_analise = [
            col
            for col in df_normalizado.columns
            if pd.api.types.is_string_dtype(df_normalizado[col])
            or df_normalizado[col].dtype == "object"
        ]
    else:
        colunas_texto_para_analise = colunas_resumo

    analise_utilidade = identificar_dados_uteis_contabeis(
        df_normalizado, colunas_texto=colunas_texto_para_analise
    )

    # Criar resumo
    resumo_contabil = criar_resumos_claros(
        df_normalizado, colunas_resumo=colunas_texto_para_analise, num_sentencas=num_sentencas_resumo
    )
    print(f"\n{resumo_contabil}")  # Imprime o resumo na tela

    # Testar a eficiência do resumo
    if resumo_esperado:
        eficiencia = testar_eficiencia_resumo(
            resumo_contabil.replace("**Resumo Contábil:**\n\n", ""), resumo_esperado
        )
        print(f"\nEficiência do resumo (similaridade): {eficiencia:.4f}")
    else:
        print("\nResumo esperado não fornecido, pulando teste de eficiência.")

    # Salvar resultados em CSV
    try:
        analise_utilidade.to_csv(nome_arquivo_csv, encoding="utf-8")
        print(f"\nResultados salvos em: {nome_arquivo_csv}")
    except Exception as e:
        print(f"Erro ao salvar em CSV: {e}")

if __name__ == "__main__":
    # Obter o nome do arquivo de saída do usuário
    nome_arquivo_saida = input("Digite o nome do arquivo CSV para salvar os resultados: ")
    if not nome_arquivo_saida:
        nome_arquivo_saida = "analise_contabil_completa.csv"  # Valor padrão se o usuário não digitar nada

    lista_de_arquivos_exemplo = [
        "relatorio_contabil_2023.csv",
        "notas_reuniao_outubro.txt",
        "balanco_patrimonial_2022.xlsx",
        "despesas_gerais.pdf",
        "demonstrativo_resultado_trimestral.csv",
    ]
    colunas_relevantes_exemplo = ["Data", "Descrição", "Valor (R$)", "Conta"]
    resumo_esperado_exemplo = "O relatório contábil detalha pagamentos e recebimentos, incluindo aluguel e vendas. Há registros de despesas e salários."

    # Criar um arquivo CSV de exemplo para teste
    data_exemplo = {
        "Data": pd.to_datetime(
            ["2023-01-01", "2023-01-01", "2023-01-02", "2023-01-02", "2023-01-03"]
        ),
        "Descrição": [
            "Pagamento de aluguel",
            "Recebimento de cliente ABC",
            "Compra de material de escritório",
            "Venda de mercadoria XYZ",
            "Salário de funcionários",
        ],
        "Valor (R$)": [1200.50, 5000.00, 350.75, 2800.00, 6000.00],
        "Conta": [
            "Despesas com aluguel",
            "Receita de vendas",
            "Despesas administrativas",
            "Receita de vendas",
            "Folha de pagamento",
        ],
        "Observação": [
            "Referente ao mês de dezembro",
            "",
            "Para uso interno",
            "NF emitida",
            "",
        ],
        "ID": [1, 2, 3, 4, 5],
        "Moeda": ["R$", "R$", "R$", "R$", "R$"],
    }
    df_exemplo = pd.DataFrame(data_exemplo)
    df_exemplo.to_csv("relatorio_contabil_2023.csv", index=False)  # Salva o arquivo de exemplo

    # Chamar a função principal
    analisar_e_salvar_dados(
        nome_arquivo_csv=nome_arquivo_saida,
        lista_arquivos=lista_de_arquivos_exemplo,
        colunas_relevantes=colunas_relevantes_exemplo,
        estrategia_tratamento_ausentes="media",
        metodo_normalizacao="minmax",
        colunas_resumo=["Descrição", "Conta", "Observação"],
        num_sentencas_resumo=2,
        resumo_esperado=resumo_esperado_exemplo,
    )

